# Import requirements

In [2]:
# !pip3 install -U bs4
# !pip3 install -U pandas

In [3]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import time
import pickle
from tqdm import tqdm_notebook as tqdm
import urllib.parse
import pandas as pd
import os

# Define functions

In [10]:
def ScrapUrl(url='https://www.zougla.gr/money/stockmarket/29'):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'}    
    req = urllib.request.Request(url,headers=headers)
    response = urllib.request.urlopen(req)
    soup = BeautifulSoup(response,'html5lib')
    shares_table_soup = soup.find('div', attrs=({'class':'shares-list-container'}))
    
    table_rows = shares_table_soup.find_all('tr')

    res = []
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text.strip() for tr in td if tr.text.strip()]
        if row:
            res.append(row)
    df = pd.DataFrame(res, columns=['Μετοχή', 'Τιμή', 'Διαφορά', '% Διαφ.', 'Ελάχ. τιμή', 'Μέγ. τιμή', 'Όγκος', 'Τζίρος'])
    return df

In [30]:
data = ScrapUrl()

In [31]:
data.describe()

,Μετοχή,Τιμή,Διαφορά,% Διαφ.,Ελάχ. τιμή,Μέγ. τιμή,Όγκος,Τζίρος
count,178,123,123,123,123,123,123,123
unique,178,106,29,88,107,104,116,122
top,ΟΛΠ,"0,22 €","0,00 €","0,00 %","0,24 €","0,22 €",20,"223,00 €"
freq,1,4,41,32,4,4,3,2


In [32]:
data.dropna(inplace=True)

In [33]:
data.sample(10)

,Μετοχή,Τιμή,Διαφορά,% Διαφ.,Ελάχ. τιμή,Μέγ. τιμή,Όγκος,Τζίρος
71,ΙΝΛΟΤ,"0,48 €","-0,01 €","-1,74 %","0,48 €","0,50 €",414120,"201447,00 €"
124,ΝΗΡ,"0,22 €","0,00 €","-0,46 %","0,21 €","0,22 €",120087,"25775,00 €"
3,ΑΒΕ,"0,32 €","0,00 €","0,00 %","0,29 €","0,32 €",5050,"1522,00 €"
151,ΠΡΟΦ,"4,21 €","0,00 €","0,00 %","4,15 €","4,25 €",26442,"111303,00 €"
159,ΣΠΙ,"0,25 €","0,00 €","0,00 %","0,25 €","0,25 €",2600,"659,00 €"
142,ΠΕΡΣ,"1,17 €","-0,03 €","-2,09 %","1,17 €","1,22 €",1791,"2131,00 €"
68,ΙΚΤΙΝ,"1,01 €","-0,01 €","-1,17 %","1,00 €","1,03 €",153579,"155412,00 €"
53,ΕΠΙΛΚ,"0,22 €","-0,01 €","-3,57 %","0,21 €","0,22 €",16853,"3657,00 €"
155,ΣΕΛΟ,"0,05 €","0,00 €","5,00 %","0,05 €","0,05 €",58843,"3098,00 €"
74,ΙΝΤΚΑ,"1,07 €","-0,02 €","-1,48 %","1,06 €","1,09 €",112287,"120859,00 €"


In [34]:
int_vals = lambda x: int(x.replace('€','').replace('%','').replace(',','.').strip())
float_vals = lambda x: float(x.replace('€','').replace('%','').replace(',','.').strip())

In [35]:
data['Τιμή'] = data['Τιμή'].apply(float_vals)
data['Διαφορά'] = data['Διαφορά'].apply(float_vals)
data['% Διαφ.'] = data['% Διαφ.'].apply(float_vals)
data['Ελάχ. τιμή'] = data['Ελάχ. τιμή'].apply(float_vals)
data['Μέγ. τιμή'] = data['Μέγ. τιμή'].apply(float_vals)
data['Τζίρος'] = data['Τζίρος'].apply(float_vals)

In [36]:
data['Όγκος'] = data['Όγκος'].apply(int_vals)

In [37]:
data.describe()

,Τιμή,Διαφορά,% Διαφ.,Ελάχ. τιμή,Μέγ. τιμή,Όγκος,Τζίρος
count,123.000000,123.000000,123.000000,123.000000,123.000000,1.230000e+02,1.230000e+02
mean,5.911626,-0.103089,-0.267398,5.867480,6.033984,2.197017e+05,3.999948e+05
std,24.228709,0.928771,3.457469,24.222586,24.966237,1.001820e+06,1.292784e+06
min,0.050000,-10.000000,-13.040000,0.050000,0.050000,1.000000e+00,1.000000e+00
25%,0.425000,-0.020000,-1.225000,0.425000,0.435000,1.063500e+03,1.125500e+03
50%,1.450000,0.000000,0.000000,1.440000,1.450000,9.049000e+03,8.116000e+03
75%,4.775000,0.010000,0.530000,4.690000,4.825000,3.997300e+04,1.160810e+05
max,262.000000,0.400000,15.790000,262.000000,270.000000,8.995580e+06,7.736819e+06


In [38]:
data.sample(10)

,Μετοχή,Τιμή,Διαφορά,% Διαφ.,Ελάχ. τιμή,Μέγ. τιμή,Όγκος,Τζίρος
18,ΑΤΤ,0.46,-0.01,-2.36,0.45,0.47,708254,324762.0
45,ΕΛΙΝ,1.92,0.00,0.00,1.86,1.92,20,37.0
49,ΕΛΣΤΡ,1.52,0.09,5.92,1.41,1.55,80212,121199.0
48,ΕΛΠΕ,9.15,-0.05,-0.54,9.07,9.20,25293,230616.0
31,ΓΕΒΚΑ,0.85,0.01,0.60,0.84,0.85,490,412.0
143,ΠΕΡΦ,3.24,0.02,0.62,3.24,3.24,500,1620.0
68,ΙΚΤΙΝ,1.01,-0.01,-1.17,1.00,1.03,153579,155412.0
105,ΜΕΝΤΙ,1.90,0.00,0.00,1.90,1.90,500,950.0
100,ΛΟΥΛΗ,3.50,-0.02,-0.57,3.48,3.50,1040,3620.0
63,ΕΦΤΖΙ,0.49,0.00,0.00,0.49,0.49,3330,1642.0


In [39]:
updown = lambda x: 1 if x>0 else (0 if x==0 else -1)
data['UpDown'] = data['Διαφορά'].apply(updown)

In [40]:
data.sample(10)

,Μετοχή,Τιμή,Διαφορά,% Διαφ.,Ελάχ. τιμή,Μέγ. τιμή,Όγκος,Τζίρος,UpDown
74,ΙΝΤΚΑ,1.07,-0.02,-1.48,1.06,1.09,112287,120859.0,-1
59,ΕΥΔΑΠ,7.82,0.02,0.26,7.64,7.85,19632,151571.0,1
55,ΕΣΥΜΒ,0.22,-0.02,-6.78,0.20,0.22,13000,2750.0,-1
72,ΙΝΤΕΚ,1.04,-0.03,-2.80,0.97,1.04,1144,1151.0,-1
91,ΚΥΡΙΟ,2.30,-0.04,-1.71,2.24,2.38,19716,46134.0,-1
109,ΜΛΣ,4.75,-0.05,-1.04,4.70,4.80,7800,36902.0,-1
148,ΠΛΑΚΡ,12.00,0.00,0.00,11.60,12.00,627,7400.0,0
62,ΕΥΡΩΒ,0.82,0.01,1.11,0.81,0.83,8995580,7358844.0,1
136,ΟΤΕ,12.33,-0.11,-0.88,12.15,12.46,458801,5653264.0,-1
167,ΤΡΑΣΤΟΡ,1.12,0.02,1.82,1.10,1.12,11940,13202.0,1
